### Home Network Assistant
---

In this notebook, we will create our first crew agent. This crew agent is the Home Network Agent. This agent will have access to a knowledge base for retrieval at runtime which will contain API specs on the home networking system. In this notebook, we will create a Knowledge base for our Home Network Agent. This agent will have access to a knowledge base for retrieval at runtime which will contain API specs on the home networking system. 

CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks.

Think of it as assembling your dream team - each member (agent) brings unique skills and expertise, collaborating seamlessly to achieve your objectives. For more information on CrewAI, view [here](https://docs.crewai.com/introduction)

**Agent code generation and execution workflow**:

1. The workflow starts with information retrieval. We create a knowledge base, and store the information from the `Home Network openAPI spec` into the knowledge base. The OpenAPI spec is provided by the user in the `data` folder.

1. This knowledge base will be wrapped within a lambda function that will be invoked based on the user query. It will `retrieve` the top `k` results from the knowledge base and send it as an input to the next step.

1. Next, a `Home Networking` assistant agent will have access to the required API specs to use to generate code. It will use the information from the Knowledge base and generate code for the given API spec, save the code and execute the code based on the parameters provided by the user.

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

#### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [1]:
# Install crew ai. For installation steps, follow the instructions here: https://docs.crewai.com/installation
!pip install 'crewai[tools]'

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: pypdf
    Found existing installation: pypdf 4.3.1
    Uninstalling pypdf-4.3.1:
      Successfully uninstalled pypdf-4.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
llama-index-legacy 0.9.48 requires tenacity<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.
deepeval 0.20.57 requires protobuf==4.25.1

In [2]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

In [3]:
# Check your boto3 version
!pip freeze | grep boto3

boto3==1.36.10


In [4]:
# import the required packages and libraries
import os
import sys
import boto3
import logging
from typing import Optional
from dotenv import load_dotenv
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
# Add the parent directory to sys.path
sys.path.append(parent_dir)
from globals import *

/Users/madhurpt/Desktop/multi-agent-code-gen-and-execution-4


In [5]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
# configure the sts client, the boto3 session and other variables
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = "us-east-1" if session.region_name is None else session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

[2025-02-10 12:50:27,984] p14938 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
import sys
sys.path.insert(0, ".")
sys.path.insert(1, "..")
# Import utility functions and helper functions for agents
from utils.utils import *

### Load the config file
--- 

Load the config file that contains information on the models, data directories, etc.

In [8]:
# Get the absolute path to the config file
# This config file contains data about the directory paths, the API specs that
# are used to generate the code, and the agent foundation models that are used to generate the code.
BASE_DIR = os.path.abspath(sys.path[1])
CONFIG_FPATH = os.path.join(BASE_DIR, CONFIG_FNAME)
config_data = load_config(CONFIG_FPATH)
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

[2025-02-10 12:50:43,518] p14938 {utils.py:61} INFO - Loading config from local file system: /Users/madhurpt/Desktop/multi-agent-code-gen-and-execution-4/config.yaml
[2025-02-10 12:50:43,529] p14938 {utils.py:64} INFO - Loaded config from local file system: {'general': {'name': 'doorbell-home-network-agentic-system', 'description': 'This agentic system shows multi-agent collaboration a home network system and a doorbell system.'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompt_templates', 'results_prefix': 'results', 'agent_instructions_prefix': 'agent_instructions', 'agent_instructions': {'doorbell_agent_instructions': 'doorbell_agent_instructions.txt', 'home_network_agent_instructions': 'home_network_agent_instructions.txt'}, 'code_gen_prompts_prefix': 'code_gen_prompts', 'code_gen_prompts': {'home_network_code_generation_prompt': 'home_network_code_generation_prompt.txt', 'doorbell_code_generation_prompt': 'doorbell_code_generation_prompt.txt'}, 'api_specs': {'home_n

## Code Generation Prompt: Bedrock prompt management
---

Let's create our sample code generation prompt by leveraging on Prompt Management for Amazon Bedrock. This is the prompt that is used by the sub agent to generate the code when it calls the generate_code tool.

In [9]:
def read_prompt_from_file(file_path: str) -> str:
    with open(file_path, 'r') as file:
        return file.read().strip()

prompt_file_path = os.path.join(
    config_data['dir_paths']['code_gen_prompts_prefix'],
    config_data['dir_paths']['code_gen_prompts'].get('home_network_code_generation_prompt')
)

absolute_prompt_fpath = os.path.join(
    parent_dir,
    prompt_file_path
)

prompt_template_code_gen: str = read_prompt_from_file(absolute_prompt_fpath)
print(f"Code generation prompt that will be saved in prompt management within Bedrock: {prompt_template_code_gen}")

Code generation prompt that will be saved in prompt management within Bedrock: With the content that you have access to, use that to generate executable python code to call the required API based on the user query and save it locally.

Your primary function is to assist users by interpreting their requests, analyzing the knowledge base content, using the required parameters in the input JSON string to generate appropriate Python code to call the home network API endpoints.

In your main function, NEVER add parameters as placeholders. Always use the parameters provided in the input JSON string and use those in the code that you generate. If there are parameters that are missing from the input JSON string
that are required to generate fully executable code, then do not generate the code and ask the user for the required parameters.

Here are your key attributes and instructions:

1. Knowledge Base Response Understanding:
- You have access to results from the knowledge base that contains 

In [10]:
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)
response = bedrock_agent.create_prompt(
    name = f"prompt-for-home-network-code-gen",
    description = "Code generation prompt template that is used by the home networking agent to generate code",
    variants = [
        {
            "name": "variantOne",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input"
                        },
                        {
                            "name": "output"
                        }
                    ],
                    "text": prompt_template_code_gen
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variantOne"
)

print(json.dumps(response, indent=2, default=str))
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}")

{
  "ResponseMetadata": {
    "RequestId": "d000d483-4740-43a0-a5a8-39e41fd6841e",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Mon, 10 Feb 2025 17:50:44 GMT",
      "content-type": "application/json",
      "content-length": "6204",
      "connection": "keep-alive",
      "x-amzn-requestid": "d000d483-4740-43a0-a5a8-39e41fd6841e",
      "x-amz-apigw-id": "Fx5iSEpjoAMEZ6Q=",
      "x-amzn-trace-id": "Root=1-67aa3c74-2303bdbf2c762f893168307c"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:218208277580:prompt/A0FEDJ0BRU",
  "createdAt": "2025-02-10 17:50:44.665124+00:00",
  "defaultVariant": "variantOne",
  "description": "Code generation prompt template that is used by the home networking agent to generate code",
  "id": "A0FEDJ0BRU",
  "name": "prompt-for-home-network-code-gen",
  "updatedAt": "2025-02-10 17:50:44.665124+00:00",
  "variants": [
    {
      "name": "variantOne",
      "templateConfiguration": {
        "text": {
          "input

In [11]:
# Now that we have a draft prompt, we can create a version from it.
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "0fdfe639-4669-411f-878f-cf817e568dad",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Mon, 10 Feb 2025 17:50:44 GMT",
      "content-type": "application/json",
      "content-length": "6095",
      "connection": "keep-alive",
      "x-amzn-requestid": "0fdfe639-4669-411f-878f-cf817e568dad",
      "x-amz-apigw-id": "Fx5iTGr2oAMEZ7g=",
      "x-amzn-trace-id": "Root=1-67aa3c74-334c6fd0524cd85636fa1fe2"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:218208277580:prompt/A0FEDJ0BRU:1",
  "createdAt": "2025-02-10 17:50:44.849122+00:00",
  "defaultVariant": "variantOne",
  "id": "A0FEDJ0BRU",
  "name": "prompt-for-home-network-code-gen",
  "updatedAt": "2025-02-10 17:50:44.849122+00:00",
  "variants": [
    {
      "name": "variantOne",
      "templateConfiguration": {
        "text": {
          "inputVariables": [
            {
              "name": "input"
            },
            {
              "name": 

## JSON Knowledge source
---

In this section of the solution we will be creating our Crew Agent, initialize the instruction for the home networking system, and then provide it with a `knowledge source`. Knowledge in CrewAI is a powerful system that allows AI agents to access and utilize external information sources during their tasks. Think of it as giving your agents a reference library they can consult while working. 

We will be creating a `JSON Knowledge Source` that will store embeddings from the `JSON` API specs that we will provide.

In [12]:
api_specs: Dict = config_data['dir_paths'].get('api_specs')
logger.info(f"API specs that are provided are as follows: {api_specs}")

[2025-02-10 12:50:45,012] p14938 {2846991394.py:2} INFO - API specs that are provided are as follows: {'home_network_api_spec': 'home_network_openapi_spec.json', 'doorbell_api_spec': 'doorbell_openapi_spec.json'}


In [77]:
!pip install --upgrade pip
!pip install -U langchain_aws

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [79]:
from langchain.schema import Document
from langchain.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

In [80]:
source_path = os.path.join('..', config_data['dir_paths']['data_prefix'], api_specs.get('home_network_api_spec'))

with open(source_path, "r", encoding="utf-8") as f:
    file_text = f.read()
    
bedrock_client = boto3.client(service_name='bedrock-runtime', 
                              region_name='us-east-1')
bedrock_embeddings = BedrockEmbeddings(model_id=config_data['model_information']['embedding_model'],
                                       client=bedrock_client)

In [81]:
documents = [Document(page_content=file_text, metadata={})]

In [82]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

db = FAISS.from_documents(docs, bedrock_embeddings)

In [88]:
retriever_topk = db.as_retriever(search_kwargs={"k": 1})
docs_topk = retriever_topk.invoke('What is the signal strength of my porch camera?')
print("\nTop k (k=1) retrieval results:")
for d in docs_topk:
    print(d.page_content)


Top k (k=1) retrieval results:
{
    "openapi": "3.0.3",
    "info": {
      "title": "SmartHome Security API",
      "description": "API specification for managing smart cameras and doorbells",
      "version": "1.0.0",
      "contact": {
        "name": "SmartHome Security Support",
        "email": "support@smarthomesecurity.example.com"
      }
    },
    "servers": [
      {
        "url": "https://api.smarthomesecurity.example.com/v1",
        "description": "Production server"
      },
      {
        "url": "https://api-staging.smarthomesecurity.example.com/v1",
        "description": "Staging server"
      }
    ],
    "tags": [
      { "name": "cameras", "description": "Camera management and control" },
      { "name": "firmware", "description": "Firmware management operations" },
      { "name": "metrics", "description": "Device metrics and statistics" },
      { "name": "recordings", "description": "Video recording management" }
    ],
    "paths": {
      "/devices/camera

### Define tools that the CrewAI agent will have access to
---

In this portion of the notebook, we create different tools that the CrewAI agent will have access to. This includes a tool to query from the knowledge base, generate code based on the user query and the API to call, save the code and finally execute the code to provide the final result to the user.

In [89]:
# Load the environment variables that are defined in the ".env" file.
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [160]:
import os
import sys
import ast
import json
import time
import boto3
import random
import logging
import tempfile
import subprocess
from crewai.tools import tool
from typing import List, Dict, Any
from crewai import LLM, Agent, Task, Crew

# Information on the code generation model
code_gen_data = config_data['code_generation_model_information']

def get_prompt_template(prompt_id: str) -> str:
    """
    Retrieve a prompt template from Bedrock prompt management.
    """
    try:
        client = boto3.client("bedrock-agent", region_name=os.environ.get("REGION", "us-east-1"))
        result = client.get_prompt(promptIdentifier=prompt_id)
        template = result["variants"][0]["templateConfiguration"]["text"]["text"]
        return template
    except Exception as e:
        logger.error(f"Error getting prompt template: {e}")
        raise

def call_bedrock(model_id: str, messages: list, temp: float, max_tokens: int, top_p: float) -> str:
    """
    Invoke Bedrock's converse API with the provided parameters.
    """
    try:
        client = boto3.client("bedrock-runtime")
        inference_config = {
            "temperature": temp,
            "maxTokens": max_tokens,
            "topP": top_p
        }
        start = time.time()
        response = client.converse(
            modelId=model_id,
            messages=messages,
            system=[{"text": "You are an expert in generating Python code for home networking APIs."}],
            inferenceConfig=inference_config
        )
        elapsed = time.time() - start
        logger.info(f"Bedrock response in {elapsed:.2f} sec")
        # Extract the generated code.
        code = response["output"]["message"]["content"][0]["text"]
        return code
    except Exception as e:
        logger.error(f"Error calling Bedrock converse: {e}")
        raise

# ─── TOOL 0: QUERY CHUNKS FROM THE IN MEMORY JSON KNOWLEDGE ───────────────────
@tool("knowledge_query_tool")
def knowledge_query_tool(query: str) -> List[Dict[str, str]]:
    """
    Searches the vector store knowledge base for relevant information based on the user's query.
    It uses a top-k retriever (with k=1) and returns a list of chunks, where each chunk is
    a dictionary with keys 'text' (the document content) and 'metadata' (optional metadata).
    """
    try:
        # Create a top-k retriever from the FAISS vector store (assumed to be defined as `db`)
        retriever_topk = db.as_retriever(search_kwargs={"k": 1})
        # Invoke the retriever with the query
        docs_topk = retriever_topk.invoke(query)
        results = [
            {"text": doc.page_content, "metadata": json.dumps(getattr(doc, "metadata", {}))}
            for doc in docs_topk
        ]
        return results
    except Exception as e:
        logger.error(f"Error querying knowledge source: {e}")
        return []


# ─── TOOL 1: CODE GENERATION VIA BEDROCK ──────────────────────────────────────
@tool("bedrock_code_generation_tool")
def bedrock_code_generation_tool(query: str, knowledge_chunks: List[Dict[str, Any]]) -> str:
    """
    Generates Python code using Bedrock, based on the user's query and knowledge chunks.
    """
    try:
        prompt_id = promptId
        template = get_prompt_template(prompt_id)
        # Combine knowledge chunks into a single string
        knowledge_content = "\n".join(chunk['text'] for chunk in knowledge_chunks)
        formatted_prompt = template.format(
            user_query=query,
            knowledge_content=knowledge_content,
            auth_token=os.getenv("HOME_NETWORK_AUTH_TOKEN")
        )
        messages = [{"role": "user", "content": [{"text": formatted_prompt}]}]
        # Get Bedrock model and inference parameters from environment variables.
        model = code_gen_data.get('code_generation_model')
        temperature = code_gen_data.get('temperature', 0.1)
        top_p = code_gen_data.get('top_p', 0.9)
        max_tokens = code_gen_data.get('max_tokens', 4096)
        code_output = call_bedrock(model, messages, temperature, max_tokens, top_p)
        return code_output
    except Exception as err:
        logger.error(f"bedrock_code_generation_tool error: {err}")
        return f"Error generating code: {err}"


# ─── TOOL 2: SAVE GENERATED CODE ─────────────────────────────────────────────
@tool("save_code_tool")
def save_code_tool(code_content: str) -> str:
    """
    Saves the provided code content to a temporary file and returns the file path.
    """
    try:
        base_dir = "generated_code"
        os.makedirs(base_dir, exist_ok=True)
        file_path = os.path.join(base_dir, f"generated_code_{int(time.time())}.py")
        with open(file_path, "w") as fp:
            fp.write(code_content)
        logger.info(f"Code saved to {file_path}")
        return file_path
    except Exception as e:
        logger.error(f"Error in save_code_tool: {e}")
        return f"Error saving code: {e}"


# ─── TOOL 3: EXECUTE GENERATED CODE ───────────────────────────────────────────
@tool("execute_code_tool")
def execute_code_tool(file_path: str) -> str:
    """
    Executes the code saved at file_path using the current Python interpreter.
    Returns a dictionary with stdout, stderr, return code, and success flag.
    """
    try:
        # For debugging, log the code that is about to be executed.
        with open(file_path, "r") as fp:
            code_text = fp.read()
        logger.info(f"Executing code:\n{code_text}")

        result = subprocess.run(
            [sys.executable, file_path],
            capture_output=True,
            text=True,
            timeout=int(os.environ.get("CODE_EXECUTION_TIMEOUT", "30")),
            env=os.environ.copy()
        )
        exec_result = f"result.stdout: {result.stdout}, stderr: {result.stderr}"
        logger.info(f"Execution result: {exec_result}")
        return exec_result
    except subprocess.TimeoutExpired:
        logger.error("Execution timed out")
        return "Execution timed out"
    except Exception as err:
        logger.error(f"Error executing code: {err}")
        return str(err)

### Create the home networking agent and Crew
---

In this portion of the solution, we will create an agent for home networking configuration assistance, that will have access to the tools defined above. We will then add this agent to the crew with the home networking knowledge base, which contains information about the API spec.

In [176]:
instructions_path: str = os.path.join(config_data['dir_paths']['agent_instructions_prefix'], 
                                      config_data['dir_paths']['agent_instructions'].get('home_network_agent_instructions'))
agent_instruction = open(os.path.join(parent_dir, instructions_path), 'r').read()
print(agent_instruction)

Human: You're a home networking management assistant that is able to help with different tasks across different home networking configurations. 
You are provided with functions that you need to answer the user question. The functions that you have access to are in the <tools></tools> tags:

<functions>
- knowledge_query_tool
- bedrock_code_generation_tool
- save_code_tool
- execute_code_tool
</functions>

For the given query '{query}', follow the steps below:

Follow the steps below in the <steps></steps> xml tags in the given order when a user asks a new question:

<steps>
1. STEP 1: First, a user will provide you with a query. Use the home network knowledge base to retrieve information about that query. 

2. STEP 2: Second, After you have the retrieved the content from the knowledge base, first validate whether the user has provided all the necessary parameters (this does not include the home network authorization token) that will be used in the 
generated code to execute it. Never a

In [177]:
from crewai.tools import tool
from crewai import LLM, Agent, Task, Crew, Process

# This is the FM used for the home networking sub agent
llm_instance = LLM(
    model=config_data['model_information']['home_network_sub_agent_model'],
    temperature=0.1,
    max_tokens=2048,
    top_p=0.9,
)

# Create the home networking sub agent along with the tools assigned to it
home_network_agent = Agent(
    role="Home Networking Configuration Assistant",
    goal=(
        agent_instruction
    ),
    backstory=(
        "I am an expert agent who looks for relevant API specs in the knowledge based on a user query, and then only I run the tools I have access to. I only run the knowledge base tool, generate code tool, save the code. Finally I will execute the code."
    ),
    tools=[knowledge_query_tool, bedrock_code_generation_tool, save_code_tool, execute_code_tool],
    verbose=True,
    memory=True,
    llm=llm_instance,
)

In [178]:
# define the tasks
knowledge_query_task = Task(
    description="Search the knowledge base for relevant information based on the user's query: '{query}'",
    expected_output="A list of knowledge chunks from the API spec relevant to the user's query.",
    agent=home_network_agent,
    output_format="list"
)

code_generation_task = Task(
    description=(
        "Using the user's query: {query} and the retrieved knowledge chunks, generate the appropriate Python code."
    ),
    expected_output="The generated Python code.",
    agent=home_network_agent,
    output_format="text",
    context=[knowledge_query_task]
)

save_code_task = Task(
    description="Save the generated Python code to a file.",
    expected_output="The file path where the code is saved.",
    agent=home_network_agent,
    output_format="text",
    context=[code_generation_task]
)

execute_code_task = Task(
    description="Execute the saved Python code using the file path from the save code task and return the output. Always run this task at the end.",
    expected_output="Output text after executing the python code.",
    agent=home_network_agent,
    output_format="text",
    context=[save_code_task]
)

In [179]:
# Initialize the manager agent that will be responsible to check for responses and coordinating the agentic workflow - this would usually be a 
# reasoning model
manager_llm = LLM(
    model=f"bedrock/{config_data['model_information']['home_network_sub_agent_model']}",
    temperature=0.1,
    timeout=120,
    max_tokens=256,
    top_p=0.9,
)

In [180]:
# Finally, create a crew that holds the agent and its task.
crew = Crew(
    agents=[home_network_agent],
    tasks=[knowledge_query_task, code_generation_task, save_code_task, execute_code_task],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=manager_llm
)

[2025-02-10 15:43:58,633] p14938 {__init__.py:537} WARNING - Overriding of current TracerProvider is not allowed


In [181]:
user_query = 'What is the signal strength of my porch camera?'
result = crew.kickoff(inputs={"query": user_query})

15:43:59 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-10 15:43:59,646] p14938 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock


# Agent: Crew Manager
## Task: Search the knowledge base for relevant information based on the user's query: 'What is the signal strength of my porch camera?'


[2025-02-10 15:44:07,930] p14938 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/converse "HTTP/1.1 200 OK"
15:44:07 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-10 15:44:07,934] p14938 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
15:44:08 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-10 15:44:08,048] p14938 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock




# Agent: Crew Manager
## Using tool: knowledge_query_tool
## Tool Input: 
"{\"query\": \"What is the signal strength of my porch camera?\"}"
## Tool Output: 
[{'text': '{\n    "openapi": "3.0.3",\n    "info": {\n      "title": "SmartHome Security API",\n      "description": "API specification for managing smart cameras and doorbells",\n      "version": "1.0.0",\n      "contact": {\n        "name": "SmartHome Security Support",\n        "email": "support@smarthomesecurity.example.com"\n      }\n    },\n    "servers": [\n      {\n        "url": "https://api.smarthomesecurity.example.com/v1",\n        "description": "Production server"\n      },\n      {\n        "url": "https://api-staging.smarthomesecurity.example.com/v1",\n        "description": "Staging server"\n      }\n    ],\n    "tags": [\n      { "name": "cameras", "description": "Camera management and control" },\n      { "name": "firmware", "description": "Firmware management operations" },\n      { "name": "metrics", "descri

[2025-02-10 15:45:10,859] p14938 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/converse "HTTP/1.1 200 OK"
15:45:10 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-10 15:45:10,867] p14938 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
[2025-02-10 15:45:17,057] p14938 {3549339764.py:50} INFO - Bedrock response in 5.67 sec
15:45:17 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-10 15:45:17,064] p14938 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-10 15:45:17,073] p14938 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


...

[2025-02-10 15:45:53,433] p14938 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/converse "HTTP/1.1 200 OK"
15:45:53 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-10 15:45:53,435] p14938 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
15:45:53 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-10 15:45:53,444] p14938 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock


 Received None or empty response from LLM call.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
# Agent: Crew Manager
## Task: Search the knowledge base for relevant information based on the user's query: 'What is the signal strength of my porch camera?'


[2025-02-10 15:45:55,624] p14938 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/converse "HTTP/1.1 200 OK"
15:45:55 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-10 15:45:55,625] p14938 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
15:45:55 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-10 15:45:55,631] p14938 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock




# Agent: Crew Manager
## Using tool: knowledge_query_tool
## Tool Input: 
"{\"query\": \"What is the signal strength of my porch camera?\"}"
## Tool Output: 
[{'text': '{\n    "openapi": "3.0.3",\n    "info": {\n      "title": "SmartHome Security API",\n      "description": "API specification for managing smart cameras and doorbells",\n      "version": "1.0.0",\n      "contact": {\n        "name": "SmartHome Security Support",\n        "email": "support@smarthomesecurity.example.com"\n      }\n    },\n    "servers": [\n      {\n        "url": "https://api.smarthomesecurity.example.com/v1",\n        "description": "Production server"\n      },\n      {\n        "url": "https://api-staging.smarthomesecurity.example.com/v1",\n        "description": "Staging server"\n      }\n    ],\n    "tags": [\n      { "name": "cameras", "description": "Camera management and control" },\n      { "name": "firmware", "description": "Firmware management operations" },\n      { "name": "metrics", "descri

KeyboardInterrupt: 